## Introduction 

The purpose of this notebook is to examine the energy requirements of optimization algorithms. We pre register the hypothesis that cache behavior is a predictor of energy intensity. 

In [1]:
using JuMP
using HiGHS
using Gurobi
using MosekTools
using Random

Random.seed!(8251)
m  = 8000
n =  8000
A = rand(m,n)
b = rand(m)
c = rand(n)


function perf(f, args)
    pid = getpid()
    cmd = `perf $args --pid=$pid`
    proc = run(pipeline(cmd, stdout=stdout, stderr=stderr); wait=false)
    try
        return f()
    finally
        flush(stdout)
        flush(stderr)
        kill(proc, Base.SIGINT)
        wait(proc)
    end
end

function perf_stat_l1(f, args = ``) 
     perf(f, `stat -e L1-dcache-load-misses,L1-dcache-loads,L1-dcache-stores,L1-icache-load-misses $args`)
end 




perf_stat_l1 (generic function with 2 methods)

## Simplex Method

In [2]:

function simplex_setup()
    model1 = Model(HiGHS.Optimizer)
    set_optimizer_attribute(model1, "presolve", "off")
    set_optimizer_attribute(model1, "parallel", "off")
    set_optimizer_attribute(model1, "solver", "simplex")
    @variable(model1, x[i=1:n] >= 0)

    @constraint(model1, A*x .<= b)

    @objective(model1, Max, c' * x)
    return model1
end 

function simplex_serial()    
    optimize!(model1)
end 

simplex_serial (generic function with 1 method)

In [11]:
model1 = simplex_setup()

A JuMP Model
Maximization problem with:
Variables: 8000
Objective function type: AffExpr
`AffExpr`-in-`MathOptInterface.LessThan{Float64}`: 8000 constraints
`VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 8000 constraints
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: HiGHS
Names registered in the model: x

In [12]:
perf_stat_l1(simplex_serial)

Running HiGHS 1.3.0 [date: 1970-01-01, git hash: e5004072b-dirty]
Copyright (c) 2022 ERGO-Code under MIT licence terms
Solving LP without presolve or with basis
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0    -1.6377626375e+06 Ph1: 8000(1.32117e+09); Du: 8000(1.63776e+06) 3s
        666     4.0696282145e-02 Pr: 1555(40.9352) 8s
       1241     8.0250057364e-03 Pr: 129(0.18045) 14s
       1428     9.4944409939e-04 Pr: 0(0) 15s
Model   status      : Optimal
Simplex   iterations: 1428
Objective value     :  9.4944409939e-04
HiGHS run time      :         15.99



 Performance counter stats for process id '4304':

     4,979,663,646      L1-dcache-load-misses     #    7.09% of all L1-dcache accesses
    70,263,545,294      L1-dcache-loads                                             
   <not supported>      L1-dcache-stores                                            
         1,523,234      L1-icache-load-misses                                       

      19.369738857 seconds time elapsed



![alt text](images/Serial_Simplex.png)

## Simplex Parallel

In [5]:

function simplex_par_setup()
    HiGHS.Highs_resetGlobalScheduler(true)
    model2 = Model(HiGHS.Optimizer)
    set_optimizer_attribute(model2, "presolve", "off")
    set_optimizer_attribute(model2, "parallel", "on")
    set_optimizer_attribute(model2, "solver", "simplex")
    @variable(model2, x[i=1:n] >= 0)

    @constraint(model2, A*x .<= b)

    @objective(model2, Max, c' * x)
    return model2
end 



function simplex_parallel()

    optimize!(model2)

end


simplex_parallel (generic function with 1 method)

In [6]:
model2 = simplex_par_setup()

A JuMP Model
Maximization problem with:
Variables: 8000
Objective function type: AffExpr
`AffExpr`-in-`MathOptInterface.LessThan{Float64}`: 8000 constraints
`VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 8000 constraints
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: HiGHS
Names registered in the model: x

In [7]:
perf_stat_l1(simplex_parallel)

## IPM Serial

In [3]:


function ipm_setup()
    model3 = Model(HiGHS.Optimizer)

    set_optimizer_attribute(model3, "presolve", "off")
    set_optimizer_attribute(model3, "parallel", "off")
    set_optimizer_attribute(model3, "solver", "ipm")

    @variable(model3, x[i=1:n] >= 0)

    @constraint(model3, A*x .<= b)

    @objective(model3, Max, c' * x)
    return model3
end

function ipm()
    optimize!(model3)
end


ipm (generic function with 1 method)

In [4]:
model3 = ipm_setup()

A JuMP Model
Maximization problem with:
Variables: 8000
Objective function type: AffExpr
`AffExpr`-in-`MathOptInterface.LessThan{Float64}`: 8000 constraints
`VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 8000 constraints
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: HiGHS
Names registered in the model: x

In [5]:
perf_stat_l1(ipm)

Running HiGHS 1.3.0 [date: 1970-01-01, git hash: e5004072b-dirty]
Copyright (c) 2022 ERGO-Code under MIT licence terms
Solving LP without presolve or with basis
IPX model has 8000 rows, 8000 columns and 64000000 nonzeros
Input
    Number of variables:                                8000
    Number of free variables:                           0
    Number of constraints:                              8000
    Number of equality constraints:                     0
    Number of matrix entries:                           64000000
    Matrix range:                                       [2e-08, 1e+00]
    RHS range:                                          [7e-05, 1e+00]
    Objective range:                                    [2e-04, 1e+00]
    Bounds range:                                       [0e+00, 0e+00]
Preprocessing
    Dualized model:                                     no
    Number of dense columns:                            0
    Range of scaling factors:                          


 Performance counter stats for process id '7342':

    53,443,655,466      L1-dcache-load-misses     #    8.77% of all L1-dcache accesses
   609,324,336,959      L1-dcache-loads                                             
   <not supported>      L1-dcache-stores                                            
        19,501,983      L1-icache-load-misses                                       

     103.218779727 seconds time elapsed



![alt text](images/ipm_seria.png)

## IPM Parallel

In [ ]:
function ipm_para()
    model3 = Model(HiGHS.Optimizer)

    set_optimizer_attribute(model3, "presolve", "off")
    set_optimizer_attribute(model3, "parallel", "on")
    set_optimizer_attribute(model3, "solver", "ipm")

    @variable(model3, x[i=1:n] >= 0)

    @constraint(model3, A*x .<= b)

    @objective(model3, Max, c' * x)
    optimize!(model3)

end

In [ ]:
ipm_para()

In [ ]:
perf_stat_l1(ipm)

In [ ]:
perf_stat_l1(simplex_serial)

In [ ]:
simplex_parallel()

In [ ]:
simplex_free(A,b,c)

In [ ]:
using Profile
using PProf
#using ProfileVega 

Profile.clear()

@profview   simplex100(A, b, c)

ProfileVega.view()

In [ ]:
using Profile
using PProf

In [1]:
using MosekTools

In [3]:

function simplex_setup()
    model1 = Model(Mosek.Optimizer)
    set_optimizer_attribute(model1, "MSK_IPAR_NUM_THREADS", 1)
    #set_optimizer_attribute(model1, "presolve", "off")
    #set_optimizer_attribute(model1, "parallel", "off")
    #set_optimizer_attribute(model1, "solver", "simplex")
    @variable(model1, x[i=1:n] >= 0)

    @constraint(model1, A*x .<= b)

    @objective(model1, Max, c' * x)
    return model1
end 

function simplex_serial()    
    # model1 = Model(HiGHS.Optimizer)
    # set_optimizer_attribute(model1, "presolve", "off")
    # set_optimizer_attribute(model1, "parallel", "off")
    # set_optimizer_attribute(model1, "solver", "simplex")
    # @variable(model1, x[i=1:n] >= 0)

    # @constraint(model1, A*x .<= b)

    # @objective(model1, Max, c' * x)
    optimize!(model1)
end 

simplex_serial (generic function with 1 method)

In [4]:
model1 = simplex_setup()

A JuMP Model
Maximization problem with:
Variables: 8000
Objective function type: AffExpr
`AffExpr`-in-`MathOptInterface.LessThan{Float64}`: 8000 constraints
`VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 8000 constraints
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: Mosek
Names registered in the model: x

In [5]:
simplex_serial()

Problem
  Name                   :                 
  Objective sense        : maximize        
  Type                   : LO (linear optimization problem)
  Constraints            : 8000            
  Affine conic cons.     : 0               
  Disjunctive cons.      : 0               
  Cones                  : 0               
  Scalar variables       : 8000            
  Matrix variables       : 0               
  Integer variables      : 0               

Optimizer started.
Presolve started.


Linear dependency checker started.
Linear dependency checker terminated.


Eliminator started.
Freed constraints in eliminator : 0


Eliminator terminated.
Eliminator - tries                  : 1                 time                   : 0.00            


Lin. dep.  - tries                  : 1                 time                   : 0.09            
Lin. dep.  - number                 : 0               
Presolve terminated. Time: 4.19    
GP based matrix reordering started.


...

19  7.5e-06  2.0e-08  7.3e-06  9.43e-01   9.763813517e-04   9.661707953e-04   2.4e-09  59.60 
20  1.9e-06  5.1e-09  1.9e-06  9.59e-01   9.576425241e-04   9.548389146e-04   6.1e-10  62.22 


21  2.2e-08  5.8e-11  2.1e-08  9.41e-01   9.495103234e-04   9.494786322e-04   7.0e-12  64.84 
22  2.3e-12  4.2e-14  2.2e-12  1.00e+00   9.494441064e-04   9.494441031e-04   7.3e-16  67.43 


Basis identification started.
Primal basis identification phase started.


Primal basis identification phase terminated. Time: 0.02
Dual basis identification phase started.
Dual basis identification phase terminated. Time: 0.00
Basis identification terminated. Time: 0.62


Optimizer terminated. Time: 69.84   

